In [4]:
pip install u8darts

  Created wheel for pymeeus: filename=PyMeeus-0.5.11-py3-none-any.whl size=730985 sha256=1ec0de79630b2d278352ae20e411ea60134bef50138017bd4eadf6da9aa34dda
  Stored in directory: c:\users\sn3028\appdata\local\pip\cache\wheels\a0\8b\b2\810ae5a6f970c8be4725353400d643c90de1c0f023a9884ee7
Successfully built pymeeus
Note: you may need to restart the kernel to use updated packages.


In [ ]:
https://towardsdatascience.com/temporal-loops-intro-to-recurrent-neural-networks-for-time-series-forecasting-in-python-b0398963dc1f

In [18]:
import pandas as pd
import numpy as np

In [5]:
pip install u8darts[fbprophet]

In [6]:
pip install u8darts[pmdarima]

In [8]:
pip install u8darts[torch]

Note: you may need to restart the kernel to use updated packages.


In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd


import torch.optim as optim
#from torch.utils.tensorboard import SummaryWriter

from darts.dataprocessing.transformers import Scaler
from darts.models import RNNModel, Theta
from darts.metrics import mape, rmse, r2_score
from darts.utils.statistics import check_seasonality, plot_acf
from darts.utils.timeseries_generation import datetime_attribute_timeseries

from darts.datasets import SunspotsDataset, AirPassengersDataset

import sys
import time
import warnings
warnings.filterwarnings("ignore")
import logging
logging.disable(logging.CRITICAL)


FC_N = 36               # forecast periods
FC_STRIDE = 10
FC_START = "19590101"   # period at which to split training and validation dataset

C:\ProgramData\Anaconda3\envs\rnn_python\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[2022-04-12 13:35:07,074] WARNING | darts.models | Support for Facebook Prophet is not available. To enable it, install "darts", "u8darts[prophet]" or "u8darts[all]" (with pip);or "u8darts-all" (with conda).


In [1]:
pip install u8darts[all]

  Using cached prophet-1.0.1.tar.gz (65 kB)
  Running setup.py clean for prophet
Failed to build prophet
    Running setup.py install for prophet: started
    Running setup.py install for prophet: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  ERROR: Command errored out with exit status 1:
   command: 'C:\ProgramData\Anaconda3\envs\rnn_python\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\sn3028\\AppData\\Local\\Temp\\pip-install-xonhgp61\\prophet_2081f47c570f4643b0f683b59268cae9\\setup.py'"'"'; __file__='"'"'C:\\Users\\sn3028\\AppData\\Local\\Temp\\pip-install-xonhgp61\\prophet_2081f47c570f4643b0f683b59268cae9\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\sn3028\AppData\Local\Temp\pip-wheel-qhvesfhz'
       cwd: C:\Users\sn3028\AppData\Local\Temp\pip-install-xonhgp61\prophet_2081f47c570f4643b0f683b59268cae9\
  Complete output (57 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib

In [208]:

EPOCH = 300       # for testing or debugging, rather than real forecasts, 
                  # set EPOCH to a low value like 3; for the real deal: 300 
                  # 300 will take as much as 30 - 50 minutes of processing time

In [4]:
## load data
ts = AirPassengersDataset().load()

series = ts  
df = ts.pd_dataframe()
df

component,#Passengers
Month,
1949-01-01,112.0
1949-02-01,118.0
1949-03-01,132.0
1949-04-01,129.0
1949-05-01,121.0
...,...
1960-08-01,606.0
1960-09-01,508.0
1960-10-01,461.0


In [5]:
# analyze its seasonality

is_seasonal, periodicity = check_seasonality(ts, max_lag=240)
dict_seas ={
    "is seasonal?":is_seasonal, 
    "periodicity (months)":f'{periodicity:.1f}', 
    "periodicity (~years)": f'{periodicity/12:.1f}'}
_ = [print(k,":",v) for k,v in dict_seas.items()]

is seasonal? : True
periodicity (months) : 12.0
periodicity (~years) : 1.0


In [6]:

# split training vs test dataset
train, val = ts.split_after(pd.Timestamp(FC_START))

In [7]:
# normalize the time series

trf = Scaler()
# fit the transformer to the training dataset
train_trf = trf.fit_transform(train)
# apply the transformer to the validation set and the complete series 
val_trf = trf.transform(val)
ts_trf = trf.transform(ts)

In [8]:
# create month and year covariate series

year_series = datetime_attribute_timeseries(
    pd.date_range(start=series.start_time(), 
        freq=ts.freq_str, 
        periods=1000),
    attribute='year', 
    one_hot=False)
year_series = Scaler().fit_transform(year_series)

month_series = datetime_attribute_timeseries(
    year_series, 
    attribute='month', 
    one_hot=True)

covariates = year_series.stack(month_series)
cov_train, cov_val = covariates.split_after(pd.Timestamp(FC_START))

In [10]:

# set up, fit, run, plot, and evaluate the RNN model

def run_RNN(flavor, ts, train, val):

    # set the model up
    model_RNN = RNNModel(
        model=flavor,
        model_name=flavor + str(" RNN"),
        input_chunk_length=periodicity,
        training_length=20,
        hidden_dim=20,
        batch_size=16,
        n_epochs=EPOCH,
        dropout=0,
        optimizer_kwargs={'lr': 1e-3},
        log_tensorboard=True,
        random_state=42,
        force_reset=True)
        
    if flavor == "RNN": flavor = "Vanilla" 

    # fit the model
    fit_it(model_RNN, train, val, flavor)

    # compute N predictions
    pred = model_RNN.predict(n=FC_N, future_covariates=covariates) 

    # plot predictions vs actual
    plot_fitted(pred, ts, flavor)

    # print accuracy metrics
    res_acc = accuracy_metrics(pred, ts)    
    print(flavor + " : ")
    _ = [print(k,":",f'{v:.4f}') for k,v in res_acc.items()]

    return [pred, res_acc]

In [ ]:
# run 3 different flavors of RNN on the time series:
flavors = ["LSTM", "GRU", "RNN"]

# call the RNN model setup for each of the 3 RNN flavors
res_flavors = [run_RNN(flv, ts_trf, train_trf, val_trf) for flv in flavors]

In [13]:

# helper function: fit the RNN model

def fit_it(model, train, val, flavor):
    t_start =  time.perf_counter()
    print("\nbeginning the training of the {0} RNN:".format(flavor))

    res = model.fit(train,
             future_covariates=covariates,
             val_series=val,
             val_future_covariates=covariates,
             verbose=True)

    res_time = time.perf_counter() - t_start
    print("training of the {0} RNN has completed:".format(flavor), f'{res_time:.2f} sec')

    return res

In [15]:
# helper function: plot the predictions

def plot_fitted(pred, act, flavor):
    plt.figure(figsize=(12,5))
    act.plot(label='actual')
    pred.plot(label='prediction')
    plt.title("RNN: {0} flavor".format(flavor) + ' | MAPE: {:.2f}%'.format(mape(pred, act)))
    plt.legend();

In [17]:
# helper function: compute accuracy metrics
   
def accuracy_metrics(pred, act):
    act2 = act.slice_intersect(pred)
    pred2 = pred.slice_intersect(act2)
    resid = pred2 - act2
    sr = resid.pd_series()
    sa = act2.pd_series()
    sp = pred2.pd_series()
    res_mape = mape(pred2, act2)
    res_r2 = r2_score(pred2, act2)
    res_rmse = rmse(pred2, act2)
    res_pe = sr / sa
    n_act = len(act2)
    res_rmspe = np.sqrt(np.sum(res_pe**2) / n_act)    # root mean square percentage error
    res_std = np.std(sr)                               # std error of the model = std deviation of the noise
    res_se = res_std / np.sqrt(n_act)                  # std error in estimating the mean
    res_sefc = np.sqrt(res_std + res_se**2)            # std error of the forecast
    
    res_accuracy = {
        "MAPE":res_mape,"RMSPE":res_rmspe, "RMSE":res_rmse, 
        "-R squared":-res_r2, "se": res_sefc}
    return res_accuracy

In [182]:
import pandas as pd
df = pd.read_csv('https://github.com/sahilnisar/.ipynb_checkpoints/raw/main/US_Forecasting_Data_1972_2021.csv', error_bad_lines=False)
df=df.set_index("date")
d=["log_gdp", "d3m_tbill", "dfed_fund", "dcpi", "d10y_bond"]
df=df.drop(d, axis=1)
df=df.iloc[4:,]
df

,dlog_gdp_yoy,unemp,3m_tbill,fedfund,cpi,manu_utilization,10y_bond
date,,,,,,,
1/1/1973,0.031658,4.933333,5.700000,6.536667,6.603333,87.702233,6.603333
4/1/1973,0.026614,4.933333,6.603333,7.816667,6.806667,87.696100,6.806667
7/1/1973,0.020241,4.800000,8.323333,10.560000,7.206667,87.502633,7.206667
10/1/1973,0.017133,4.766667,7.500000,9.996667,6.753333,88.285867,6.753333
1/1/1974,0.002765,5.133333,7.616667,9.323333,7.053333,86.648667,7.053333
...,...,...,...,...,...,...,...
10/1/2020,-0.009941,6.766667,0.093333,0.090000,0.863333,74.182667,0.863333
1/1/2021,0.002369,6.200000,0.050000,0.080000,1.316667,74.733533,1.316667
4/1/2021,0.050096,5.900000,0.026667,0.070000,1.593333,75.761367,1.593333


In [191]:
from darts import TimeSeries
import pandas as pd
df = pd.read_csv('https://github.com/sahilnisar/.ipynb_checkpoints/raw/main/US_Forecasting_Data_1972_2021.csv', error_bad_lines=False)
d=["log_gdp", "d3m_tbill", "dfed_fund", "dcpi", "d10y_bond"]
df=df.drop(d, axis=1)
df=df.iloc[4:,]
df=TimeSeries.from_dataframe(df, 'date')
df

<TimeSeries (DataArray) (date: 196, component: 7, sample: 1)>
array([[[3.16584180e-02],
        [4.93333333e+00],
        [5.70000000e+00],
        ...,
        [6.60333333e+00],
        [8.77022333e+01],
        [6.60333333e+00]],

       [[2.66135790e-02],
        [4.93333333e+00],
        [6.60333333e+00],
        ...,
        [6.80666667e+00],
        [8.76961000e+01],
        [6.80666667e+00]],

       [[2.02414510e-02],
        [4.80000000e+00],
        [8.32333333e+00],
        ...,
...
        ...,
        [1.59333333e+00],
        [7.57613667e+01],
        [1.59333333e+00]],

       [[2.09681890e-02],
        [5.10000000e+00],
        [4.66666670e-02],
        ...,
        [1.32333333e+00],
        [7.63843667e+01],
        [1.32333333e+00]],

       [[2.34841570e-02],
        [4.23333333e+00],
        [5.33333330e-02],
        ...,
        [1.53666667e+00],
        [7.73605667e+01],
        [1.53666667e+00]]])
Coordinates:
  * date       (date) datetime64[ns] 1973-01-01 1973-04-01 ... 2021-10-01
  * component  (component) object 'dlog_gdp_yoy' 'unemp' ... '10y_bond'
Dimensions without coordinates: sample

In [195]:
FC_N = 16               # forecast periods
FC_STRIDE = 10
FC_START = "20180101"   # period at which to split training and validation dataset
# split training vs test dataset
train, val = df['dlog_gdp_yoy'].split_after(pd.Timestamp(FC_START))

In [219]:
# normalize the time series

trf = Scaler()
# fit the transformer to the training dataset
train_trf = trf.fit_transform(train)
# apply the transformer to the validation set and the complete series 
val_trf = trf.transform(val)
df_trf = trf.transform(df['dlog_gdp_yoy'])

In [220]:
# create month and year covariate series

year_series = datetime_attribute_timeseries(
    pd.date_range(start=series.start_time(), 
        freq=df['dlog_gdp_yoy'].freq_str, 
        periods=1000),
    attribute='year', 
    one_hot=False)
year_series = Scaler().fit_transform(year_series)



In [221]:
# run 3 different flavors of RNN on the time series:
flavors = ["LSTM", "GRU", "RNN"]

# call the RNN model setup for each of the 3 RNN flavors
res_flavors = [run_RNN(flv, df_trf, train_trf, val_trf) for flv in flavors]


beginning the training of the LSTM RNN:


ValueError: __len__() should return >= 0

In [ ]:
series = ts  
df = ts.pd_dataframe()
df
df.index = pd.to_datetime(df['date'])
df.index = pd.to_datetime(df.index, format = '%m/%d/%Y').strftime('%Y/%m/%d')
gdp=ts.to_frame()
## load data
ts = AirPassengersDataset()
type(ts)

month_series = datetime_attribute_timeseries(
    year_series, 
    attribute='month', 
    one_hot=True)

covariates = year_series.stack(month_series)
cov_train, cov_val = covariates.split_after(pd.Timestamp(FC_START))
